# 🔒 Python Encapsulation Masterclass

## 📚 Table of Contents
1. Understanding Encapsulation
2. Access Modifiers in Python
3. Properties and Decorators
4. Data Protection Patterns
5. Advanced Encapsulation Techniques
6. Real-World Applications
7. Best Practices

## 🎯 Learning Objectives
After completing this notebook, you will:
- Master Python's encapsulation mechanisms
- Understand private and protected attributes
- Use properties and decorators effectively
- Implement data protection patterns
- Apply encapsulation in real-world scenarios

## 1. Understanding Encapsulation 🔒

Encapsulation is the bundling of data and methods that operate on that data within a single unit (class), and restricting access to the internal details.

```
    🔒 Class Boundary
    ┌─────────────────────┐
    │   Private Data      │
    │   ┌──────────┐     │
    │   │ __data   │     │
    │   └──────────┘     │
    │                    │
    │   Public Interface │
    │   ┌──────────┐     │
    │   │ get_data()│     │
    │   │ set_data()│     │
    │   └──────────┘     │
    └─────────────────────┘
```

In [ ]:
# Basic Encapsulation Example
class BankAccount:
    def __init__(self, balance):
        self.__balance = balance  # Private attribute
    
    def get_balance(self):
        return self.__balance
    
    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
            return True
        return False
    
    def withdraw(self, amount):
        if 0 < amount <= self.__balance:
            self.__balance -= amount
            return True
        return False

# Using the class
account = BankAccount(1000)
print(f"Initial balance: ${account.get_balance()}")
account.deposit(500)
print(f"After deposit: ${account.get_balance()}")
account.withdraw(200)
print(f"After withdrawal: ${account.get_balance()}")

# This will not work directly:
# print(account.__balance)  # AttributeError

## 2. Access Modifiers in Python 🏷️

Python uses naming conventions for access control:
- Public: `name`
- Protected: `_name`
- Private: `__name`

Let's explore each type:

In [ ]:
class Employee:
    def __init__(self, name, salary):
        self.name = name          # Public
        self._salary = salary     # Protected
        self.__id = 12345         # Private
    
    def _calculate_bonus(self):   # Protected method
        return self._salary * 0.1
    
    def __generate_id(self):      # Private method
        return f"EMP{self.__id}"
    
    def get_details(self):        # Public method
        return {
            'name': self.name,
            'id': self.__generate_id(),
            'bonus': self._calculate_bonus()
        }

# Using the class
emp = Employee("Alice", 50000)
print(f"Name (public): {emp.name}")
print(f"Salary (protected): {emp._salary}")  # Accessible but shouldn't be used
# print(emp.__id)  # This will raise an AttributeError
print(f"Details: {emp.get_details()}")

## 3. Properties and Decorators 🎀

Properties provide a way to customize access to attributes:

In [ ]:
class Temperature:
    def __init__(self, celsius=0):
        self.__celsius = celsius
    
    @property
    def celsius(self):
        return self.__celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:  # Absolute zero
            raise ValueError("Temperature below absolute zero!")
        self.__celsius = value
    
    @property
    def fahrenheit(self):
        return (self.__celsius * 9/5) + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        self.__celsius = (value - 32) * 5/9

# Using properties
temp = Temperature(25)
print(f"Celsius: {temp.celsius}°C")
print(f"Fahrenheit: {temp.fahrenheit}°F")

temp.fahrenheit = 100
print(f"New Celsius: {temp.celsius}°C")

# This will raise an error:
# temp.celsius = -300  # ValueError

## 4. Data Protection Patterns 🛡️

Let's explore various patterns for protecting data:

In [ ]:
from datetime import datetime

class User:
    def __init__(self, username, password):
        self.__username = username
        self.__password = self.__encrypt_password(password)
        self.__login_attempts = 0
        self.__last_login = None
    
    def __encrypt_password(self, password):
        # Simple encryption (don't use in production!)
        return ''.join(chr(ord(c) + 1) for c in password)
    
    def __decrypt_password(self, encrypted):
        return ''.join(chr(ord(c) - 1) for c in encrypted)
    
    def check_password(self, password):
        self.__login_attempts += 1
        if self.__login_attempts > 3:
            raise ValueError("Account locked! Too many attempts")
        
        if self.__decrypt_password(self.__password) == password:
            self.__login_attempts = 0
            self.__last_login = datetime.now()
            return True
        return False
    
    @property
    def last_login(self):
        return self.__last_login
    
    @property
    def username(self):
        return self.__username

# Using the User class
user = User("alice", "secret123")
print(f"Login successful: {user.check_password('secret123')}")
print(f"Last login: {user.last_login}")

## 5. Advanced Encapsulation Techniques 🚀

### 5.1 Descriptor Protocol

In [ ]:
class ValidString:
    def __init__(self, minlen=0, maxlen=100):
        self.minlen = minlen
        self.maxlen = maxlen
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(f"__{self.__class__.__name__}")
    
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError("Value must be a string")
        if not self.minlen <= len(value) <= self.maxlen:
            raise ValueError(f"String length must be between {self.minlen} and {self.maxlen}")
        instance.__dict__[f"__{self.__class__.__name__}"] = value

class Person:
    name = ValidString(minlen=2, maxlen=30)
    address = ValidString(maxlen=200)
    
    def __init__(self, name, address):
        self.name = name
        self.address = address

# Using the descriptor
person = Person("Alice", "123 Main St")
print(f"Name: {person.name}")
print(f"Address: {person.address}")

# These will raise errors:
# person.name = "A"  # ValueError: too short
# person.name = 123  # TypeError: not a string

### 5.2 Context Managers for Resource Protection

In [ ]:
class DatabaseConnection:
    def __init__(self, connection_string):
        self.__connection_string = connection_string
        self.__connection = None
    
    def __enter__(self):
        print(f"Connecting to database...")
        self.__connection = f"Connected to {self.__connection_string}"
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Closing database connection...")
        self.__connection = None
    
    def execute_query(self, query):
        if not self.__connection:
            raise RuntimeError("Not connected to database")
        print(f"Executing query: {query}")

# Using the context manager
with DatabaseConnection("mysql://localhost:3306/mydb") as db:
    db.execute_query("SELECT * FROM users")

# This will raise an error:
# db.execute_query("SELECT 1")  # RuntimeError: Not connected

## 6. Real-World Applications 🌍

### 6.1 E-Commerce System

In [ ]:
from decimal import Decimal
from datetime import datetime, timedelta

class Product:
    def __init__(self, name, price, stock):
        self.__name = name
        self.__price = Decimal(str(price))
        self.__stock = stock
        self.__sales_history = []
    
    @property
    def name(self):
        return self.__name
    
    @property
    def price(self):
        return self.__price
    
    @price.setter
    def price(self, value):
        if value < 0:
            raise ValueError("Price cannot be negative")
        self.__price = Decimal(str(value))
    
    def __record_sale(self, quantity, timestamp):
        self.__sales_history.append({
            'quantity': quantity,
            'timestamp': timestamp,
            'price': self.__price
        })
    
    def sell(self, quantity):
        if quantity <= 0:
            raise ValueError("Quantity must be positive")
        if quantity > self.__stock:
            raise ValueError("Not enough stock")
        
        self.__stock -= quantity
        self.__record_sale(quantity, datetime.now())
        return quantity * self.__price
    
    def get_sales_report(self, days=30):
        cutoff = datetime.now() - timedelta(days=days)
        recent_sales = [sale for sale in self.__sales_history 
                       if sale['timestamp'] > cutoff]
        
        total_quantity = sum(sale['quantity'] for sale in recent_sales)
        total_revenue = sum(sale['quantity'] * sale['price'] for sale in recent_sales)
        
        return {
            'total_quantity': total_quantity,
            'total_revenue': total_revenue,
            'average_price': total_revenue / total_quantity if total_quantity > 0 else 0
        }

# Using the Product class
laptop = Product("Gaming Laptop", 1299.99, 10)
print(f"Selling 2 {laptop.name}s")
total = laptop.sell(2)
print(f"Total: ${total}")

# Wait a bit and sell more
laptop.sell(1)
report = laptop.get_sales_report(days=7)
print(f"Sales report: {report}")

### 6.2 Secure Configuration Manager

In [ ]:
import json
from typing import Any, Dict

class ConfigManager:
    __instance = None
    __config: Dict[str, Any] = {}
    __sensitive_keys = set()
    
    def __new__(cls):
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
        return cls.__instance
    
    def __init__(self):
        # Initialize only once
        if not self.__config:
            self.__load_config()
    
    def __load_config(self):
        # Simulate loading from a file
        self.__config = {
            'database_url': 'postgresql://localhost:5432/mydb',
            'api_key': 'secret_key_123',
            'debug_mode': True
        }
        self.__sensitive_keys = {'api_key', 'database_url'}
    
    def get(self, key: str, default: Any = None) -> Any:
        return self.__config.get(key, default)
    
    def set(self, key: str, value: Any) -> None:
        self.__config[key] = value
    
    def __str__(self) -> str:
        safe_config = {}
        for key, value in self.__config.items():
            if key in self.__sensitive_keys:
                safe_config[key] = '***HIDDEN***'
            else:
                safe_config[key] = value
        return json.dumps(safe_config, indent=2)

# Using the ConfigManager
config = ConfigManager()
print("Configuration:")
print(config)

print(f"\nDebug mode: {config.get('debug_mode')}")
config.set('debug_mode', False)
print(f"Updated debug mode: {config.get('debug_mode')}")

## 7. Practice Exercises 🎯

### Exercise 1: Create a Password Manager

In [ ]:
class PasswordManager:
    def __init__(self, master_password):
        # TODO: Implement secure password storage
        pass
    
    def add_password(self, service, password):
        # TODO: Add encrypted password
        pass
    
    def get_password(self, service, master_password):
        # TODO: Retrieve and decrypt password
        pass

### Exercise 2: Implement a Secure Logger

In [ ]:
class SecureLogger:
    def __init__(self, log_file):
        # TODO: Initialize secure logging
        pass
    
    def log(self, message, level='INFO'):
        # TODO: Implement secure logging
        pass
    
    def get_logs(self, start_date=None, end_date=None):
        # TODO: Retrieve logs with date filtering
        pass

### Exercise 3: Create a Bank Account System

In [ ]:
class BankAccount:
    def __init__(self, account_number, initial_balance):
        # TODO: Initialize account with proper encapsulation
        pass
    
    def transfer(self, other_account, amount):
        # TODO: Implement secure transfer
        pass
    
    def get_statement(self, start_date, end_date):
        # TODO: Generate account statement
        pass

## 🎯 Final Challenge Project

Create a complete Health Records System that demonstrates:
- Secure patient data storage
- Access control levels (doctor, nurse, admin)
- Audit logging
- Data encryption
- HIPAA compliance features

Requirements:
1. Patient data must be fully encapsulated
2. Different access levels for different roles
3. Complete audit trail of all access
4. Secure data transmission simulation
5. Data validation and sanitization

In [ ]:
# Start of your implementation
class HealthRecord:
    pass

class Patient:
    pass

class MedicalStaff:
    pass

class AuditLog:
    pass

class HealthRecordSystem:
    pass

## 📚 Best Practices

1. Always use private attributes when dealing with sensitive data
2. Implement proper validation in property setters
3. Use descriptors for reusable validation logic
4. Implement proper error handling
5. Document your encapsulation decisions
6. Use context managers for resource management
7. Implement proper access controls

## 🎉 Summary

- Encapsulation protects data integrity
- Python provides multiple levels of access control
- Properties and descriptors provide clean interfaces
- Context managers help with resource management
- Proper encapsulation is crucial for security

Keep practicing these concepts to build secure and maintainable systems!
